In [6]:
"""The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.

S is the starting position; there is a pipe on this tile,
but the sketch doesn't show what shape the pipe has.
"""

example_1_1 =""".....
.F-7.
.|.|.
.L-J.
.....
"""

example_1_2 =""".....
.S-7.
.|.|.
.L-J.
.....
"""

example_1_3 ="""-L|F7
7S-7|
L|7||
-L-J|
L|-JF
"""

example_2_1 ="""..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

example_2_2 ="""7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
"""

example_3_1="""...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
"""

example_3_2="""..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........
"""
example_4=""".F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
"""

In [21]:
import numpy as np
from collections import deque

starting_pos = (0,0)
# Define the possible pipe types
pipe_types = {
    '|': [(-1, 0), (1, 0)],
    '-': [(0, -1), (0, 1)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, 1), (1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, -1), (-1, 0)],
    '.': [],
    'S': [(0, 1), (0, -1), (1, 0), (-1, 0)]
}

def split_input(input_data):
    return [list(row) for row in input_data.splitlines()]

def verify_symbol(mirrored_connections, local_grid_array, x_row, x_col):
    """Verify that the connections are valid for the given pipe type"""
    # Check each position relative to 'X'
    for dr, dc in mirrored_connections:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if (dr,dc) not in pipe_types[local_grid_array[new_row, new_col][0]]:
                return None
        else:
            return None
    return mirrored_connections

def check_pipe(x_row, x_col, local_grid_array, original_char, debug=False):
    """
    Determine if a pipe is valid based on a 3x3 grid.
    Valid pipes have exactly two connections and is the expected
    character, or 'S'
    """
    # Check each position relative to 'X'
    valid_pipes = 0
    is_starting_pos = False
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_row, new_col = x_row + dr, x_col + dc
        if 0 <= new_row < local_grid_array.shape[0] and 0 <= new_col < local_grid_array.shape[1]:
            if debug:
                print(f"New row: {dr}, new col: {dc}, char: {local_grid_array[new_row, new_col]}")
            if (dr,dc) in pipe_types[local_grid_array[new_row, new_col][0]]:
                if debug:
                    print(f"Found valid pipe {local_grid_array[new_row, new_col]}")
                valid_pipes += 1
    # If there's exactly two valid pipes next to 'X', return True, else False
    if valid_pipes >= 2:
        if original_char != 'S':
            mirrored_connections = [(-dr, -dc) for dr, dc in pipe_types[original_char]]
            return verify_symbol(mirrored_connections, local_grid_array, x_row, x_col), is_starting_pos
        else:
            # Try all possible combinations of connections
            for guess_char, mirrored_connections in pipe_types.items():
                mirrored_connections = [(-dr, -dc) for dr, dc in mirrored_connections]
                result = verify_symbol(mirrored_connections, local_grid_array, x_row, x_col)
                if result:
                    print(f"Found starting position! Assuming 'S' is {guess_char}")
                    is_starting_pos = True
                    return result, is_starting_pos
    if debug:
        print(f"Invalid pipe at {x_row}, {x_col}, when checking: {original_char}, found {valid_pipes} valid pipes")
    return None, is_starting_pos

def traverse_loop(start, pipes):
    """
    Iteratively traverse the loop starting from 'start', counting steps until returning to 'start'.
    Returns and all positions visited along the way.
    """
    current_pos = start
    prev_pos = None
    steps = 0
    visited = set()
    visited.add(start)
    while True:
        next_pos = None
        for neighbor in pipes[current_pos]:
            if neighbor != prev_pos or len(pipes[current_pos]) == 1:
                visited.add(neighbor)
                next_pos = neighbor
                break

        if next_pos == start and current_pos != start:
            # Completed the loop and returned to the starting position
            return steps, visited

        prev_pos = current_pos
        current_pos = next_pos
        steps += 1

def show_grid(valid_pipes, content):
    # Update the grid with the new pipes, take shape  and mark pipes with emojis
    new_grid = np.zeros(content.shape, dtype=str)
    new_grid.fill('🚫')
    for pos, neighbors in valid_pipes.items():
        new_grid[pos] = '🔗'
    # mark everywhere in content which is ground, '.' with '🌱'
    new_grid[content == '.'] = '🌱'

    # configure numpy to print max columns and rows to 15
    np.set_printoptions(linewidth=150)
    display(new_grid)



with open('adventofcode.com_2023_day_10_input.txt', 'r') as f:
    input_string = f.read()

content = np.array(split_input(example_3_2))
display(content)
# Loop through every character, take a 3x3 grid around it, and check if it's a valid pipe
valid_pipes = {}
for row in range(content.shape[0]):
    for col in range(content.shape[1]):
        debug = False
        # Handle edge cases by creating partial grids
        grid_slice = content[max(0, row-1):min(content.shape[0], row+2), max(0, col-1):min(content.shape[1], col+2)].copy()
        # Determine the position of 'X' in the grid slice
        x_row, x_col = min(1, row), min(1, col)
        # Save the original character and mark the current position with 'X'
        original_char = grid_slice[x_row, x_col]
        if original_char == '-':
            debug = False
        grid_slice[x_row, x_col] = 'X'
        neighbors, found_start = check_pipe(x_row, x_col,grid_slice, original_char,debug)
        if neighbors:
            # print(f"Found valid pipe at {row}, {col}")
            valid_pipes[(row, col)] = [(row+dr, col+dc) for dr, dc in neighbors]
        if found_start:
            print(f"Setting starting position to {row}, {col}")
            starting_pos = (row, col)
# display(valid_pipes)
print(f"Found {len(valid_pipes)} valid pipes")
if starting_pos == (0,0):
    raise Exception("No starting position found!")
total_steps, visited = traverse_loop(starting_pos, valid_pipes)
print(f"Total steps in the loop: {total_steps}")
print(f"Half of the steps: {total_steps // 2 + 1}")

# Filter out the pipes that were not visited
valid_pipes = {k: v for k, v in valid_pipes.items() if k in visited}
show_grid(valid_pipes, content)

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', 'S', '-', '-', '-', '-', '-', '-', '7', '.'],
       ['.', '|', 'F', '-', '-', '-', '-', '7', '|', '.'],
       ['.', '|', '|', '.', '.', '.', '.', '|', '|', '.'],
       ['.', '|', '|', '.', '.', '.', '.', '|', '|', '.'],
       ['.', '|', 'L', '-', '7', 'F', '-', 'J', '|', '.'],
       ['.', '|', '.', '.', '|', '|', '.', '.', '|', '.'],
       ['.', 'L', '-', '-', 'J', 'L', '-', '-', 'J', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

Found starting position! Assuming 'S' is F
Setting starting position to 1, 1
Found 44 valid pipes
Total steps in the loop: 43
Half of the steps: 22


array([['🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱'],
       ['🌱', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🌱'],
       ['🌱', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🌱'],
       ['🌱', '🔗', '🔗', '🌱', '🌱', '🌱', '🌱', '🔗', '🔗', '🌱'],
       ['🌱', '🔗', '🔗', '🌱', '🌱', '🌱', '🌱', '🔗', '🔗', '🌱'],
       ['🌱', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🌱'],
       ['🌱', '🔗', '🌱', '🌱', '🔗', '🔗', '🌱', '🌱', '🔗', '🌱'],
       ['🌱', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🌱'],
       ['🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱', '🌱']], dtype='<U1')

In [49]:
import numpy as np

def create_barrier_grid(pipe_types, grid):
    """
    Create a grid marking the barriers based on the pipe_types.
    """
    barrier_grid = np.full(grid.shape, 0)  # Default to passable

    rows, cols = grid.shape
    for row in range(rows):
        for col in range(cols):
            pipe = grid[row, col]
            if pipe in pipe_types:
                # Marking barriers according to pipe type
                for dr, dc in pipe_types[pipe]:
                    if 0 <= row - dr < rows and 0 <= col - dc < cols:
                        barrier_grid[row - dr, col - dc] |= (1 << ((dr + 1) * 3 + (dc + 1)))

    return barrier_grid

def flood_fill(grid, barrier_grid):
    """
    Perform a flood fill from all edges of the grid, respecting barriers.
    """
    rows, cols = grid.shape
    stack = [(0, y) for y in range(cols)] + [(rows-1, y) for y in range(cols)] + \
            [(x, 0) for x in range(rows)] + [(x, cols-1) for x in range(rows)]

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]

    while stack:
        debug=False
        x, y = stack.pop()
        if x == 6 and y == 6:
            debug = True
        if 0 <= x < rows and 0 <= y < cols and grid[x, y] != 'o':
            for dr, dc in directions:
                if debug:
                    print(f"Checking {x+dr}, {y+dc}: {content[x+dr,y+dc]}  for {x}, {y}")
                if 0 <= x + dr < rows and 0 <= y + dc < cols:
                    barrier_bit = (1 << ((dr + 1) * 3 + (dc + 1)))
                    if not (barrier_grid[x+dr, y+dc] & barrier_bit):
                        if debug:
                            print(f"{x+dr}, {y+dc} is passable, adding to stack, barrier bit: {barrier_bit:08b}, barrier grid: {barrier_grid[x+dr, y+dc]:08b}")
                        stack.append((x + dr, y + dc))
            grid[x, y] = 'o'  # Mark as visited


def count_enclosed_tiles(grid):
    """
    Count the number of unmarked '.' tiles after flood fill.
    """
    return np.sum(grid == '.')

def split_input(input_data):
    return [list(row) for row in input_data.splitlines()]

example_3_2="""..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........
"""

pipe_types = {
    '|': [(-1, 0), (1, 0)],
    '-': [(0, -1), (0, 1)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, 1), (1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, -1), (-1, 0)],
    '.': []
}

valid_pipes = {(1, 1): [(1, 2), (2, 1)],
 (1, 2): [(1, 3), (1, 1)],
 (1, 3): [(1, 4), (1, 2)],
 (1, 4): [(1, 5), (1, 3)],
 (1, 5): [(1, 6), (1, 4)],
 (1, 6): [(1, 7), (1, 5)],
 (1, 7): [(1, 8), (1, 6)],
 (1, 8): [(1, 7), (2, 8)],
 (2, 1): [(3, 1), (1, 1)],
 (2, 2): [(2, 3), (3, 2)],
 (2, 3): [(2, 4), (2, 2)],
 (2, 4): [(2, 5), (2, 3)],
 (2, 5): [(2, 6), (2, 4)],
 (2, 6): [(2, 7), (2, 5)],
 (2, 7): [(2, 6), (3, 7)],
 (2, 8): [(3, 8), (1, 8)],
 (3, 1): [(4, 1), (2, 1)],
 (3, 2): [(4, 2), (2, 2)],
 (3, 7): [(4, 7), (2, 7)],
 (3, 8): [(4, 8), (2, 8)],
 (4, 1): [(5, 1), (3, 1)],
 (4, 2): [(5, 2), (3, 2)],
 (4, 7): [(5, 7), (3, 7)],
 (4, 8): [(5, 8), (3, 8)],
 (5, 1): [(6, 1), (4, 1)],
 (5, 2): [(5, 3), (4, 2)],
 (5, 3): [(5, 4), (5, 2)],
 (5, 4): [(5, 3), (6, 4)],
 (5, 5): [(5, 6), (6, 5)],
 (5, 6): [(5, 7), (5, 5)],
 (5, 7): [(5, 6), (4, 7)],
 (5, 8): [(6, 8), (4, 8)],
 (6, 1): [(7, 1), (5, 1)],
 (6, 4): [(7, 4), (5, 4)],
 (6, 5): [(7, 5), (5, 5)],
 (6, 8): [(7, 8), (5, 8)],
 (7, 1): [(7, 2), (6, 1)],
 (7, 2): [(7, 3), (7, 1)],
 (7, 3): [(7, 4), (7, 2)],
 (7, 4): [(7, 3), (6, 4)],
 (7, 5): [(7, 6), (6, 5)],
 (7, 6): [(7, 7), (7, 5)],
 (7, 7): [(7, 8), (7, 6)],
 (7, 8): [(7, 7), (6, 8)]}

content = np.array(split_input(example_3_2))

# Create a barrier grid based on valid_pipes
barrier_grid = create_barrier_grid(pipe_types, content)

# Use the content grid for the flood fill
grid = content.copy()
display(grid)
flood_fill(grid, barrier_grid)
tiles_enclosed = count_enclosed_tiles(grid)
print(f"Number of tiles enclosed by the loop: {tiles_enclosed}")
show_grid(valid_pipes, grid)
barrier_grid

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', 'S', '-', '-', '-', '-', '-', '-', '7', '.'],
       ['.', '|', 'F', '-', '-', '-', '-', '7', '|', '.'],
       ['.', '|', '|', '.', '.', '.', '.', '|', '|', '.'],
       ['.', '|', '|', '.', '.', '.', '.', '|', '|', '.'],
       ['.', '|', 'L', '-', '7', 'F', '-', 'J', '|', '.'],
       ['.', '|', '.', '.', '|', '|', '.', '.', '|', '.'],
       ['.', 'L', '-', '-', 'J', 'L', '-', '-', 'J', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']], dtype='<U1')

Checking 5, 6: -  for 6, 6
5, 6 is passable, adding to stack, barrier bit: 00000010, barrier grid: 00101000
Checking 7, 6: -  for 6, 6
7, 6 is passable, adding to stack, barrier bit: 10000000, barrier grid: 00101000
Checking 6, 5: |  for 6, 6
6, 5 is passable, adding to stack, barrier bit: 00001000, barrier grid: 10000010
Checking 6, 7: .  for 6, 6
6, 7 is passable, adding to stack, barrier bit: 00100000, barrier grid: 00000000
Checking 5, 5: F  for 6, 6
5, 5 is passable, adding to stack, barrier bit: 00000001, barrier grid: 10100000
Checking 5, 7: J  for 6, 6
5, 7 is passable, adding to stack, barrier bit: 00000100, barrier grid: 00001010
Checking 7, 5: L  for 6, 6
7, 5 is passable, adding to stack, barrier bit: 01000000, barrier grid: 00100010
Checking 7, 7: -  for 6, 6
7, 7 is passable, adding to stack, barrier bit: 100000000, barrier grid: 00101000
Number of tiles enclosed by the loop: 0


array([['🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫'],
       ['🚫', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🚫'],
       ['🚫', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🚫'],
       ['🚫', '🔗', '🔗', '🚫', '🚫', '🚫', '🚫', '🔗', '🔗', '🚫'],
       ['🚫', '🔗', '🔗', '🚫', '🚫', '🚫', '🚫', '🔗', '🔗', '🚫'],
       ['🚫', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🚫'],
       ['🚫', '🔗', '🚫', '🚫', '🔗', '🔗', '🚫', '🚫', '🔗', '🚫'],
       ['🚫', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🔗', '🚫'],
       ['🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫', '🚫']], dtype='<U1')

array([[  0, 128,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 32, 160,  40,  40,  40,  40,  40,  40, 136,   0],
       [  0, 130, 160,  40,  40,  40,  40, 136, 130,   0],
       [  0, 130, 130,   0,   0,   0,   0, 130, 130,   0],
       [  0, 130, 130,   0,   0,   0,   0, 130, 130,   0],
       [  0, 130,  34,  40, 136, 160,  40,  10, 130,   0],
       [  0, 130,   0,   0, 130, 130,   0,   0, 130,   0],
       [  0,  34,  40,  40,  10,  34,  40,  40,  10,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [37]:
content[6,6]

'.'

In [32]:
# North (Up): Bit 1
# North-East (Up-Right): Bit 2
# East (Right): Bit 3
# South-East (Down-Right): Bit 4
# South (Down): Bit 5
# South-West (Down-Left): Bit 6
# West (Left): Bit 7
# North-West (Up-Left): Bit 8
print(f"{130:08b}")

10000010
